## Contrarian Strategy

Importing dependencies

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots

Fetching the data source and converting to pandas dataframe

In [2]:
data = pd.read_csv(filepath_or_buffer='../../resources/intraday.csv', parse_dates=['time'], index_col='time')

Getting details about the asset data by analysing the dataframe

In [3]:
data.head(10)

,Close
time,
2009-12-31,1.432706
2010-01-01,1.438994
2010-01-04,1.442398
2010-01-05,1.436596
2010-01-06,1.440403
2010-01-07,1.431803
2010-01-08,1.441109
2010-01-11,1.451126
2010-01-12,1.447660


In [4]:
data.tail(10)

,Close
time,
2021-12-20,1.124354
2021-12-21,1.127752
2021-12-22,1.128757
2021-12-23,1.132888
2021-12-24,1.132734
2021-12-27,1.132426
2021-12-28,1.133003
2021-12-29,1.131478
2021-12-30,1.136015


In [5]:
data.Close.to_frame()

,Close
time,
2009-12-31,1.432706
2010-01-01,1.438994
2010-01-04,1.442398
2010-01-05,1.436596
2010-01-06,1.440403
...,...
2021-12-27,1.132426
2021-12-28,1.133003
2021-12-29,1.131478


Getting more statistical insight into the dataset

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3127 entries, 2009-12-31 to 2021-12-31
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Close   3127 non-null   float64
dtypes: float64(1)
memory usage: 48.9 KB


In [7]:
data.describe()

,Close
count,3127.000000
mean,1.219630
std,0.107936
min,1.039047
25%,1.124366
50%,1.190079
75%,1.317046
max,1.484406


Since the only attribute we need is closing price
we modify the data frame

In [8]:
data = data.Close.to_frame()

In [9]:
data.head(10)

,Close
time,
2009-12-31,1.432706
2010-01-01,1.438994
2010-01-04,1.442398
2010-01-05,1.436596
2010-01-06,1.440403
2010-01-07,1.431803
2010-01-08,1.441109
2010-01-11,1.451126
2010-01-12,1.447660


We plot the closing price of our asset

In [10]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=data.index, y=data.Close, name='Close'))

fig.update_layout(title='EUR/USD', xaxis_title='Time', yaxis_title='Price')

fig.show()

In [11]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=data.loc['2018'].index, y=data.loc['2018'].Close, name='Close'))

fig.update_layout(title='EUR/USD', xaxis_title='Time', yaxis_title='Price')

fig.show()

We proceed to define baseline and compute returns !needs improvement!

In [12]:
data['returns'] = np.log(data.div(data.shift(1)))

In [13]:
data.dropna(inplace=True)

In [14]:
data

,Close,returns
time,,
2010-01-01,1.438994,0.004379
2010-01-04,1.442398,0.002363
2010-01-05,1.436596,-0.004031
2010-01-06,1.440403,0.002647
2010-01-07,1.431803,-0.005989
...,...,...
2021-12-27,1.132426,-0.000272
2021-12-28,1.133003,0.000510
2021-12-29,1.131478,-0.001347


Defining a simple contrarian strategy (window = 3)

In [15]:
window = 3

In [16]:
data['returns'].rolling(window).mean()

time
2010-01-01         NaN
2010-01-04         NaN
2010-01-05    0.000904
2010-01-06    0.000326
2010-01-07   -0.002457
                ...   
2021-12-27    0.001082
2021-12-28    0.000034
2021-12-29   -0.000370
2021-12-30    0.001055
2021-12-31   -0.000147
Name: returns, Length: 3126, dtype: float64

In [17]:
data['position'] = -np.sign(data['returns'].rolling(window).mean()) # contrarian (minus sign)

In [18]:
data

,Close,returns,position
time,,,
2010-01-01,1.438994,0.004379,NaN
2010-01-04,1.442398,0.002363,NaN
2010-01-05,1.436596,-0.004031,-1.0
2010-01-06,1.440403,0.002647,-1.0
2010-01-07,1.431803,-0.005989,1.0
...,...,...,...
2021-12-27,1.132426,-0.000272,-1.0
2021-12-28,1.133003,0.000510,-1.0
2021-12-29,1.131478,-0.001347,1.0


In [19]:
data['strategy'] = data.position.shift(1) * data['returns']

In [20]:
data.dropna(inplace=True)

In [21]:
data

,Close,returns,position,strategy
time,,,,
2010-01-06,1.440403,0.002647,-1.0,-0.002647
2010-01-07,1.431803,-0.005989,1.0,0.005989
2010-01-08,1.441109,0.006478,-1.0,0.006478
2010-01-11,1.451126,0.006927,-1.0,-0.006927
2010-01-12,1.447660,-0.002391,-1.0,0.002391
...,...,...,...,...
2021-12-27,1.132426,-0.000272,-1.0,0.000272
2021-12-28,1.133003,0.000510,-1.0,-0.000510
2021-12-29,1.131478,-0.001347,1.0,0.001347


In [22]:
data[['returns', 'strategy']].sum()

returns    -0.237846
strategy   -0.036154
dtype: float64

In [23]:
data['creturns'] = data['returns'].cumsum().apply(np.exp)
data['cstrategy'] = data['strategy'].cumsum().apply(np.exp)

In [24]:
data

,Close,returns,position,strategy,creturns,cstrategy
time,,,,,,
2010-01-06,1.440403,0.002647,-1.0,-0.002647,1.002650,0.997357
2010-01-07,1.431803,-0.005989,1.0,0.005989,0.996664,1.003347
2010-01-08,1.441109,0.006478,-1.0,0.006478,1.003142,1.009868
2010-01-11,1.451126,0.006927,-1.0,-0.006927,1.010114,1.002897
2010-01-12,1.447660,-0.002391,-1.0,0.002391,1.007702,1.005299
...,...,...,...,...,...,...
2021-12-27,1.132426,-0.000272,-1.0,0.000272,0.788270,0.956868
2021-12-28,1.133003,0.000510,-1.0,-0.000510,0.788672,0.956380
2021-12-29,1.131478,-0.001347,1.0,0.001347,0.787610,0.957670


In [25]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=data.index, y=data.creturns, name='Returns (Baseline)'))
fig.add_trace(go.Scatter(x=data.index, y=data.cstrategy, name='Returns (Strategy)'))

fig.update_layout(title='EUR/USD', xaxis_title='Time', yaxis_title='Price')

fig.show()

Taking trading costs into consideration

In [26]:
ptc = 0.00007

In [27]:
data['trades'] = data.position.diff().fillna(0).abs()

In [28]:
data

,Close,returns,position,strategy,creturns,cstrategy,trades
time,,,,,,,
2010-01-06,1.440403,0.002647,-1.0,-0.002647,1.002650,0.997357,0.0
2010-01-07,1.431803,-0.005989,1.0,0.005989,0.996664,1.003347,2.0
2010-01-08,1.441109,0.006478,-1.0,0.006478,1.003142,1.009868,2.0
2010-01-11,1.451126,0.006927,-1.0,-0.006927,1.010114,1.002897,0.0
2010-01-12,1.447660,-0.002391,-1.0,0.002391,1.007702,1.005299,0.0
...,...,...,...,...,...,...,...
2021-12-27,1.132426,-0.000272,-1.0,0.000272,0.788270,0.956868,0.0
2021-12-28,1.133003,0.000510,-1.0,-0.000510,0.788672,0.956380,0.0
2021-12-29,1.131478,-0.001347,1.0,0.001347,0.787610,0.957670,2.0


In [29]:
data.trades.value_counts()

0.0    2306
2.0     817
Name: trades, dtype: int64

In [30]:
data['strategy_net'] = data.strategy - data.trades * ptc

In [31]:
data['cstrategy_net'] = data.strategy_net.cumsum().apply(np.exp)

In [32]:
data

,Close,returns,position,strategy,creturns,cstrategy,trades,strategy_net,cstrategy_net
time,,,,,,,,,
2010-01-06,1.440403,0.002647,-1.0,-0.002647,1.002650,0.997357,0.0,-0.002647,0.997357
2010-01-07,1.431803,-0.005989,1.0,0.005989,0.996664,1.003347,2.0,0.005849,1.003207
2010-01-08,1.441109,0.006478,-1.0,0.006478,1.003142,1.009868,2.0,0.006338,1.009586
2010-01-11,1.451126,0.006927,-1.0,-0.006927,1.010114,1.002897,0.0,-0.006927,1.002617
2010-01-12,1.447660,-0.002391,-1.0,0.002391,1.007702,1.005299,0.0,0.002391,1.005017
...,...,...,...,...,...,...,...,...,...
2021-12-27,1.132426,-0.000272,-1.0,0.000272,0.788270,0.956868,0.0,0.000272,0.853807
2021-12-28,1.133003,0.000510,-1.0,-0.000510,0.788672,0.956380,0.0,-0.000510,0.853372
2021-12-29,1.131478,-0.001347,1.0,0.001347,0.787610,0.957670,2.0,0.001207,0.854403


We plot returns with and without trading costs

In [33]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=data.index, y=data.creturns, name='Returns (Baseline)'))
fig.add_trace(go.Scatter(x=data.index, y=data.cstrategy, name='Returns (Strategy)'))
fig.add_trace(go.Scatter(x=data.index, y=data.cstrategy_net, name='Returns (Strategy + trading costs)'))

fig.update_layout(title='EUR/USD', xaxis_title='Time', yaxis_title='Price')

fig.show()

In [34]:
data[['returns', 'strategy_net']].mean() * (252)

returns        -0.019192
strategy_net   -0.012147
dtype: float64

In [35]:
data[['returns', 'strategy_net']].std() * np.sqrt(252)

returns         0.086889
strategy_net    0.086415
dtype: float64

Now we make use of a comprehensive backtester which implements Contrarian startegy and optimize our parameters to maximize returns

In [36]:
from MomentumContrarianBacktester import MomentumContrarianBacktester as MCB

In [37]:
testMCB = MCB(symbol='EUR/USD', start='2010', end='2021', tc=0.00007, flag='c')

In [38]:
testMCB

MomentumContrarianBacktester(symbol = EUR/USD, start = 2010, end = 2021, flag = c)

In [39]:
testMCB.data

,Close,returns
time,,
2010-01-04,1.442398,0.002363
2010-01-05,1.436596,-0.004031
2010-01-06,1.440403,0.002647
2010-01-07,1.431803,-0.005989
2010-01-08,1.441109,0.006478
...,...,...
2021-12-27,1.132426,-0.000272
2021-12-28,1.133003,0.000510
2021-12-29,1.131478,-0.001347


In [40]:
testMCB.test_strategy()

(1.789343, 1.00419)

In [41]:
testMCB.results

,Close,returns,position,strategy,trades,strategy_net,creturns,cstrategy,cstrategy_net
time,,,,,,,,,
2010-01-05,1.436596,-0.004031,1.0,0.004031,0.0,0.004031,0.995978,1.004039,1.004039
2010-01-06,1.440403,0.002647,-1.0,0.002647,2.0,0.002507,0.998617,1.006700,1.006559
2010-01-07,1.431803,-0.005989,1.0,0.005989,2.0,0.005849,0.992655,1.012746,1.012463
2010-01-08,1.441109,0.006478,-1.0,0.006478,2.0,0.006338,0.999107,1.019329,1.018901
2010-01-11,1.451126,0.006927,-1.0,-0.006927,0.0,-0.006927,1.006051,1.012292,1.011867
...,...,...,...,...,...,...,...,...,...
2021-12-27,1.132426,-0.000272,1.0,-0.000272,0.0,-0.000272,0.785100,1.773390,1.415316
2021-12-28,1.133003,0.000510,-1.0,0.000510,2.0,0.000370,0.785500,1.774294,1.415839
2021-12-29,1.131478,-0.001347,1.0,0.001347,2.0,0.001207,0.784442,1.776686,1.417550


We find the number of trades made

In [42]:
testMCB.results.trades.value_counts()

2.0    1597
0.0    1491
1.0      36
Name: trades, dtype: int64

In [43]:
testMCB.plot_results()

We give a set of boundaries to optimize our parameters within

window_range describes the range of days over which need to be taken into consideration

In [44]:
window_range = (1, 200)

In [45]:
testMCB.optimize_parameter(window_range=window_range)

(1, 1.789343)

In [46]:
testMCB.test_strategy(window=44)

(0.982261, 0.148558)

Set of parameters which give maximum returns and a plot of returns for the same

In [47]:
testMCB.plot_results()

In [48]:
#Return
testMCB.results[['returns', 'strategy_net']].mean() * (252)

returns        -0.014876
strategy_net   -0.003983
dtype: float64

In [49]:
#Risk
testMCB.results[['returns', 'strategy_net']].std() * np.sqrt(252)

returns         0.086702
strategy_net    0.086582
dtype: float64